In [1]:
import findspark
import os

In [3]:
os.environ['SPARK_HOME'] = '/Users/arthur/spark'

In [4]:
findspark.init()

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName('SparkStudy')\
    .getOrCreate() 

21/12/13 23:58:15 WARN Utils: Your hostname, Arthurs-Macbook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.12 instead (on interface en0)
21/12/13 23:58:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/13 23:58:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
spark

In [7]:
#Exemplo de dataframe
data = [('Zeca','35'),('Joao','25'),('Maria','30')]
colNames = ['name','age']
df = spark.createDataFrame(data,colNames)

In [8]:
df.show()

+-----+---+
| name|age|
+-----+---+
| Zeca| 35|
| Joao| 25|
|Maria| 30|
+-----+---+



In [9]:
df.toPandas()

,name,age
0,Zeca,35
1,Joao,25
2,Maria,30


In [10]:
path = '/Users/arthur/Desktop/SparkStudies/content/empresas'

In [11]:
empresas = spark.read.csv(path,sep=';',header=True)

AnalysisException: Unable to infer schema for CSV. It must be specified manually.

In [ ]:
empresas.count()

Poderia usar o padrão dessa forma:

    path = '/Users/arthur/Desktop/SparkStudies/content/empresas/part-0000?-5487.csv'>
    path = '/Users/arthur/Desktop/SparkStudies/content/empresas/part-0000*'>


In [ ]:
path_estabelecimentos = '/Users/arthur/Desktop/SparkStudies/content/estabelecimentos/part-0000*'
path_socios = '/Users/arthur/Desktop/SparkStudies/content/socios/part-0000?-1ab6459b-5a3c-4294-83b8-316c171f0202-c000.csv'

estabelecimentos = spark.read.csv(path_estabelecimentos,sep=';',inferSchema=True)
socios = spark.read.csv(path_socios,sep=';',inferSchema=True)

In [ ]:
estabelecimentos.count()

In [ ]:
socios.count()

## Operações Básicas

In [ ]:
empresas.limit(5).toPandas()

In [ ]:
empresasColNames = ['cnpj_basico', 'razao_social_nome_empresarial', 'natureza_juridica', 'qualificacao_do_responsavel', 'capital_social_da_empresa', 'porte_da_empresa', 'ente_federativo_responsavel']

In [ ]:
for index, colName in enumerate(empresasColNames):
    empresas = empresas.withColumnRenamed(f"_c{index}",colName)

empresas.columns


In [ ]:
estabsColNames = ['cnpj_basico', 'cnpj_ordem', 'cnpj_dv', 'identificador_matriz_filial', 'nome_fantasia', 'situacao_cadastral', 'data_situacao_cadastral', 'motivo_situacao_cadastral', 'nome_da_cidade_no_exterior', 'pais', 'data_de_inicio_atividade', 'cnae_fiscal_principal', 'cnae_fiscal_secundaria', 'tipo_de_logradouro', 'logradouro', 'numero', 'complemento', 'bairro', 'cep', 'uf', 'municipio', 'ddd_1', 'telefone_1', 'ddd_2', 'telefone_2', 'ddd_do_fax', 'fax', 'correio_eletronico', 'situacao_especial', 'data_da_situacao_especial']
sociosColNames = ['cnpj_basico', 'identificador_de_socio', 'nome_do_socio_ou_razao_social', 'cnpj_ou_cpf_do_socio', 'qualificacao_do_socio', 'data_de_entrada_sociedade', 'pais', 'representante_legal', 'nome_do_representante', 'qualificacao_do_representante_legal', 'faixa_etaria']

In [ ]:
estabelecimentos.limit(5).toPandas()

In [ ]:
socios.limit(5).toPandas()

In [ ]:
for index, colName in enumerate(estabsColNames):
    estabelecimentos = estabelecimentos.withColumnRenamed(f"_c{index}",colName)

for index, colName in enumerate(sociosColNames):
    socios = socios.withColumnRenamed(f"_c{index}",colName)

## Analisando os Dados

In [ ]:
empresas.printSchema()

In [ ]:
socios.printSchema()

data_de_entrada_sociedade está como inteiro, temos que fazer a transformação para data

Vamos realizar a transformação dos dados de string ou int para double ou data

In [ ]:
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql import functions as f

In [ ]:
empresas.printSchema()

In [ ]:
empresas.limit(5).toPandas()

In [ ]:
empresas = empresas.withColumn('capital_social_da_empresa',f.regexp_replace('capital_social_da_empresa',',','.'))
empresas.limit(5).toPandas()

In [ ]:
empresas = empresas.withColumn('capital_social_da_empresa', empresas['capital_social_da_empresa'].cast(DoubleType()))
empresas.limit(5).toPandas()

Com datas...

In [ ]:
df = spark.createDataFrame([(20200924,),(20201022,),(20210215,)],['data'])
df.toPandas()

In [ ]:
df.printSchema()

In [ ]:
df = df.withColumn('data', f.to_date(df.data.cast(StringType()),'yyyyMMdd'))
df.toPandas()

In [ ]:
estabelecimentos.printSchema()

In [ ]:
estabelecimentos = estabelecimentos \
    .withColumn(
        'data_situacao_cadastral',
        f.to_date(estabelecimentos.data_situacao_cadastral.cast(StringType()),'yyyyMMdd')
    )\
    .withColumn(
        'data_de_inicio_atividade',
        f.to_date(estabelecimentos.data_de_inicio_atividade.cast(StringType()),'yyyyMMdd')
    )\
    .withColumn(
        'data_da_situacao_especial',
        f.to_date(estabelecimentos.data_da_situacao_especial.cast(StringType()),'yyyyMMdd')
    )

In [ ]:
estabelecimentos.limit(3).toPandas()

In [ ]:
socios.limit(3).toPandas()

In [ ]:
socios = socios.withColumn('data_de_entrada_sociedade', f.to_date(socios.data_de_entrada_sociedade.cast(StringType()),'yyyyMMdd'))

# Seleções e Consultas

In [ ]:
empresas\
    .select('*')\
    .show(5, True) # Segundo parâmetro é truncated

In [ ]:
empresas\
    .select('natureza_juridica','porte_da_empresa','capital_social_da_empresa')\
    .show(5)

In [ ]:
socios\
    .select('nome_do_socio_ou_razao_social','faixa_etaria',f.year('data_de_entrada_sociedade').alias('ano_de_entrada')) \
    .show(5)

In [ ]:
estabelecimentos\
    .select('nome_fantasia',\
        'municipio',\
        f.year('data_de_inicio_atividade').alias('ano_de_inicio_da_atividade'),\
        f.month('data_de_inicio_atividade').alias('mês_de_inicio_da_atividade'))\
    .show(5)

### Exercícios

In [ ]:
## Exercício

data = [
    ('GISELLE PAULA GUIMARAES CASTRO', 15),
    ('ELAINE GARCIA DE OLIVEIRA', 22),
    ('JOAO CARLOS ABNER DE LOURDES', 43),
    ('MARTA ZELI FERREIRA', 24),
    ('LAUDENETE WIGGERS ROEDER', 51)
]
colNames = ['nome', 'idade']
df = spark.createDataFrame(data, colNames)
df.show(truncate=False)

In [ ]:
df\
    .select(\
        f.concat_ws(
            ', ',\
            f.split('nome',' ')[1],\
            f.substring_index('nome',' ',-1))\
        .alias('nome'),\
        'idade'
    )\
    .show(5, truncate=False)

## Valores Nulos

Ter atenção quanto aos tipos nulos, NaN é Not a Number e Null é um valor vazio.

In [ ]:
socios.limit(5).toPandas()

In [ ]:
socios.limit(5).show(5)

In [ ]:
socios.select([f.count(f.when(f.isnull(col),1)).alias(col) for col in socios.columns]).toPandas()

In [ ]:
socios.printSchema()

Para substituir valores Nulos ou NaN

In [ ]:
socios.na.fill(0).limit(5).toPandas()

In [ ]:
socios.na.fill('-').limit(5).toPandas()

## Organizando Dados

In [ ]:
socios\
    .select('nome_do_socio_ou_razao_social','faixa_etaria',f.year('data_de_entrada_sociedade').alias('ano_de_entrada')) \
    .show(5, False)

In [ ]:
socios\
    .select('nome_do_socio_ou_razao_social','faixa_etaria',f.year('data_de_entrada_sociedade').alias('ano_de_entrada')) \
    .orderBy('ano_de_entrada',ascending=True)\
    .show(5, False)

In [ ]:
socios\
    .select('nome_do_socio_ou_razao_social','faixa_etaria',f.year('data_de_entrada_sociedade').alias('ano_de_entrada')) \
    .orderBy('ano_de_entrada',ascending=False)\
    .show(5, False)

In [ ]:
socios\
    .select('nome_do_socio_ou_razao_social','faixa_etaria',f.year('data_de_entrada_sociedade').alias('ano_de_entrada')) \
    .orderBy(['ano_de_entrada', 'faixa_etaria'],ascending=[False, True])\
    .show(10, False)

### Exercícios

In [ ]:
data = [
    ('CARMINA RABELO', 4, 2010), 
    ('HERONDINA PEREIRA', 6, 2009), 
    ('IRANI DOS SANTOS', 12, 2010), 
    ('JOAO BOSCO DA FONSECA', 3, 2009), 
    ('CARLITO SOUZA', 1, 2010), 
    ('WALTER DIAS', 9, 2009), 
    ('BRENO VENTUROSO', 1, 2009), 
    ('ADELINA TEIXEIRA', 5, 2009), 
    ('ELIO SILVA', 7, 2010), 
    ('DENIS FONSECA', 6, 2010)
]
colNames = ['nome', 'mes', 'ano']
df = spark.createDataFrame(data, colNames)
df.show(truncate=False)

In [ ]:
df.select('*').orderBy(['ano','mes'],ascending=[False,False]).show()

## Filtrando os Dados

Usa-se o método Where e Filter

In [ ]:
empresas\
    .where('capital_social_da_empresa == 50')\
    .show(5)

In [ ]:
socios\
    .select('nome_do_socio_ou_razao_social')\
    .filter(socios.nome_do_socio_ou_razao_social.startswith("RODRIGO"))\
    .filter(socios.nome_do_socio_ou_razao_social.endswith("SOUZA"))\
    .limit(10)\
    .toPandas()

### Exercícios

In [ ]:
df.show(4)

In [ ]:
df.filter('ano=2009').filter('mes<=6').show()

In [ ]:
df.filter((df.ano == 2009) & (df.mes <= 6)).show()

## Método Like

In [ ]:
df = spark.createDataFrame([('RESTAURANTE DO RUI',),('Juca Restaurantes ltda', ),('Joca Restaurante', )],['data'])
df.toPandas()

In [ ]:
df\
    .where(
        f.upper(
            df.data
        ).like('%RESTAURANTE%')
    )\
    .toPandas()

In [ ]:
df\
    .where(
        f.upper(
            df.data
        ).like('RESTAURANTE%')
    )\
    .toPandas()

In [ ]:
df\
    .where(
        f.upper(
            df.data
        ).like('%RESTAURANTE')
    )\
    .toPandas()

In [ ]:
empresas\
    .select('razao_social_nome_empresarial','natureza_juridica','porte_da_empresa','capital_social_da_empresa')\
    .filter(f.upper(empresas['razao_social_nome_empresarial']).like('%RESTAURANTE%'))\
    .limit(10)\
    .toPandas()

### Exercícios

In [ ]:
data = [
    ('CARMINA RABELO', 4, 2010), 
    ('HERONDINA PEREIRA', 6, 2009), 
    ('IRANI DOS SANTOS', 12, 2010), 
    ('JOAO BOSCO DA FONSECA', 3, 2009), 
    ('CARLITO SOUZA', 1, 2010), 
    ('WALTER DIAS', 9, 2009), 
    ('BRENO VENTUROSO', 1, 2009), 
    ('ADELINA TEIXEIRA', 5, 2009), 
    ('ELIO SILVA', 7, 2010), 
    ('DENIS FONSECA', 6, 2010)
]
colNames = ['nome', 'mes', 'ano']
df = spark.createDataFrame(data, colNames)
df.show(truncate=False)

In [ ]:
df\
    .filter(df.nome.like('C%'))\
    .show()

# Agregações e Junções
## Sumarizando Dados

In [ ]:
socios\
    .select(f.year('data_de_entrada_sociedade').alias('ano_de_entrada'))\
    .where('ano_de_entrada >= 2010')\
    .groupBy('ano_de_entrada')\
    .count()\
    .orderBy('ano_de_entrada',ascending=True)\
    .show()

In [ ]:
empresas\
    .select('cnpj_basico','porte_da_empresa','capital_social_da_empresa')\
    .groupBy('porte_da_empresa')\
    .agg(
        f.avg('capital_social_da_empresa').alias('capital_social_medio'),
        f.count('cnpj_basico').alias('frequencia')
    )\
    .orderBy('porte_da_empresa',ascending=True)\
    .show()

In [ ]:
empresas\
    .select('capital_social_da_empresa')\
    .summary()\
    .show()

## Juntando Dataframes

In [ ]:
idades = spark.createDataFrame(
    [
        ('CARLOS', 15), 
        ('IVO', 14), 
        ('MÁRCIA', 16), 
        ('LEILA', 17), 
        ('LETÍCIA', 14)
    ],
    ['nomes', 'idades']
)

notas = spark.createDataFrame(
    [
        ('CARLOS', 10), 
        ('MÁRCIA', 1), 
        ('LEILA', 10), 
        ('BRENO', 7), 
        ('LETÍCIA', 9)
    ],
    ['nomes', 'notas']
)

In [ ]:
idades.join(notas,'nomes',how='outer')\
    .sort('nomes')\
    .show()

In [ ]:
full_df = idades.join(notas,'nomes',how='outer')\
    .sort('nomes')

In [ ]:
full_df.union(
    full_df.select(
        f.lit('Total').alias('data'),
        f.sum(full_df.idades).alias('idades_teste'),
        f.sum(full_df.notas).alias('notas_teste')
    )
)

## SparkSQL 

In [ ]:
empresas.createOrReplaceTempView('empresasView')

In [ ]:
spark.sql("SELECT * FROM empresasView").show(5)

In [ ]:
spark\
    .sql(
        """
            SELECT *
            FROM empresasView
            WHERE natureza_juridica = 2240
        """
).show(5)

# Formas de Armazenamento

In [ ]:
empresas.write.csv(
    path = 'empresas.csv',
    mode='overwrite',
    sep=";",
    header=True
)

In [ ]:
df.write.parquet(
    path='df.parquet',
    mode='overwrite'
)

In [13]:
df.coalesce(1).write.csv(
    path = 'df_coalesce1.csv',
    mode='overwrite',
    sep=";",
    header=True
)

In [15]:
df.coalesce(1).write.parquet(
    path = 'df_coalesce1_parquet',
    mode='overwrite',
    partitionBy = 'age'
)

In [ ]:
spark.stop()